In [77]:
from torch import optim
from utils import *
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
import torch
import torch.nn as nn
from TorchCRF import CRF
from typing import List, Tuple, Optional
import math


# import evaluation metrics 
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score

torch.manual_seed(42)

In [78]:
if torch.cuda.is_available():
    device = torch.device('cuda')

device

device(type='cuda')

In [79]:
# class ConstrainedCRF(CRF):
#     def __init__(self, num_tags):
#         super(ConstrainedCRF, self).__init__(num_tags)

#     def decode(self, emissions: torch.Tensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]], mask: Optional[torch.ByteTensor] = None) -> List[List[int]]:
#         """Find the most likely tag sequence using Viterbi algorithm.

#         Args:
#             emissions (`~torch.Tensor`): Emission score tensor of size
#                 ``(seq_length, batch_size, num_tags)`` if ``batch_first`` is ``False``,
#                 ``(batch_size, seq_length, num_tags)`` otherwise.
#             mask (`~torch.ByteTensor`): Mask tensor of size ``(seq_length, batch_size)``
#                 if ``batch_first`` is ``False``, ``(batch_size, seq_length)`` otherwise.

#         Returns:
#             List of list containing the best tag sequence for each batch.
#         """
#         self._validate(emissions, mask=mask)
#         if mask is None:
#             mask = emissions.new_ones(emissions.shape[:2], dtype=torch.uint8)

#         if self.batch_first:
#             emissions = emissions.transpose(0, 1)
#             mask = mask.transpose(0, 1)

#         return self._viterbi_decode(emissions, mask, constraints)

#     def _viterbi_decode(self, emissions: torch.FloatTensor, mask: torch.ByteTensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]]) -> List[List[int]]:
#             print("Viterbi Decode")
#             '''
#                 override the viterbi decode function to include the constraints
#             '''
#             assert emissions.dim() == 3 and mask.dim() == 2 # (sequence_length, batch_size, num_tags)
#             assert emissions.shape[:2] == mask.shape # (sequence_length, batch_size)
#             assert emissions.size(2) == self.num_tags
#             assert mask[0].all()

#             sequence_length, batch_size = mask.shape

#             constrained_transitions = self.transitions.clone()
#             constrainted_satrt_transitions = self.start_transitions.clone()
#             for constraint in constraints:
#                 if constraint[0] == -1:
#                     constrainted_satrt_transitions[constraint[1]] = INF
#                 else:
#                     constrained_transitions[constraint[0], constraint[1]] = INF



#             # start transition and first emission
#             # tensor of size (batch_size, num_tags)
#             # constraints for start transition
#             # score = self.start_transitions + emissions[0] 
#             score = constrainted_satrt_transitions + emissions[0] 

#             backpointers = []

#             for i in range(1, sequence_length):
#                 # Broadcast viterbi score for every possible next tag
#                 # shape: (batch_size, num_tags, 1)
#                 broadcast_score = score.unsqueeze(2)
                
                    
#                 # Broadcast emission score for every possible current tag
#                 # shape: (batch_size, 1, num_tags)
#                 broadcast_emission = emissions[i].unsqueeze(1)

#                 # Compute the score tensor of size (batch_size, num_tags, num_tags) where
#                 # for each sample, entry at row i and column j stores the score of the best
#                 # tag sequence so far that ends with transitioning from tag i to tag j and emitting
#                 # shape: (batch_size, num_tags, num_tags)
#                 # next_score = broadcast_score + self.transitions + broadcast_emission
#                 next_score = broadcast_score + constrained_transitions + broadcast_emission

#                 # print("Next Score Shape:", next_score[0].shape)

#                 # Apply the constraints
#                 # for constraint in constraints:
#                 #     # print("Constraint:", constraint)
#                 #     next_score[0][constraint[0], constraint[1]] = INF

                
#                 # Find the maximum score over all possible current tag
#                 # shape: (batch_size, num_tags)
#                 next_score, indices = next_score.max(dim=1)

#                 # Set score to the next score if this timestep is valid (mask == 1)
#                 # and save the index that produces the next score
#                 # shape: (batch_size, num_tags)
#                 score = torch.where(mask[i].unsqueeze(1), next_score, score)
#                 backpointers.append(indices)
            
            
#             # End transition score
#             # shape: (batch_size, num_tags)
#             score += self.end_transitions


#             # Now, compute the best path for each sample

#             # shape: (batch_size,)
#             seq_ends = mask.long().sum(dim=0) - 1
#             best_tags_list = []

#             for idx in range(batch_size):
#                 # Find the tag which maximizes the score at the last timestep; this is our best tag
#                 # for the last timestep
#                 _, best_last_tag = score[idx].max(dim=0)
#                 best_tags = [best_last_tag.item()]

#                 # We trace back where the best last tag comes from, append that to our best tag
#                 # sequence, and trace it back again, and so on
#                 for hist in reversed(backpointers[:seq_ends[idx]]):
#                     best_last_tag = hist[idx][best_tags[-1]]
#                     best_tags.append(best_last_tag.item())

#                 # Reverse the order because we start from the last timestep
#                 best_tags.reverse()
#                 best_tags_list.append(best_tags)

#             return best_tags_list



# class BiLSTMCRF(nn.Module):
#     def __init__(self, vocab_size, word_embedding_dim, intent_embedding_dim, hidden_dim, output_dim, number_of_intents, index_to_tag):
        
#         super(BiLSTMCRF, self).__init__()
#         # hyperparameters
#         self.word_embedding_dim = word_embedding_dim
#         self.inten_embedding_dim = intent_embedding_dim
#         self.hidden_dim = hidden_dim
#         self.vocab_size = vocab_size
#         self.output_dim = output_dim
#         self.index_to_tag = index_to_tag

#         # model layers
#         self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
#         self.intent_embedding = nn.Embedding(number_of_intents, intent_embedding_dim)
#         self.lstm = nn.LSTM(word_embedding_dim + intent_embedding_dim, hidden_dim // 2, bidirectional=True, dropout=0.2)
#         self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
#         self.crf = ConstrainedCRF(output_dim)

#     def __create_constraints(self, mask, intent):
#         constraints = []
#         one_indices = torch.where(mask == 1)[0] # tags that are for the intent
#         zero_indices = torch.where(mask == 0)[0] # tags that are not for the intent
        
#         print(one_indices)
#         print(zero_indices)

#         for i in one_indices:
#             # add constraints for the tags that are related to the intent
#             # the transition from an intent tag to a non-intent tag is not allowed
#             for j in zero_indices:
#                 constraints.append((i, j))
#                 constraints.append((j, i))
            
#             # constraints that the tag if O can't be followed by I
#             # but I can be followed by O 
#             if self.index_to_tag[i.item()] == 'O':
#                 for j in one_indices:
#                     if self.index_to_tag[j.item()][0] == 'I':
#                         constraints.append((i, j))
        
#             # add constraints that if the tag begins with B then the next tag can't begin with B of the same 
#             # if tag begins with B then it can't be followed by B of the same entity
#             # or I of a different entity
#             # it can be followed by I of same entity, O or B of a different entity
#             # add constraints that if the tag begins with B then the next tag cant begin with I of a different entity
#             if self.index_to_tag[i.item()][0] == 'B':
#                 constraints.append((i, i))
#                 for j in one_indices:
#                     if self.index_to_tag[j.item()][0] == 'I' and self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#                         constraints.append((i, j))
                        
#             # if self.index_to_tag[i.item()][0] == 'B':
#             #     for j in one_indices:
#             #         if self.index_to_tag[j.item()][0] == 'I' and self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#             #             constraints.append((i, j))

#             # create start constraints
#             # if the tag starts with I then it can't be the first tag
#             if self.index_to_tag[i.item()][0] == 'I':
#                 constraints.append((-1, i))
#                 for j in one_indices:
#                     if self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#                         constraints.append((i, j))
#                         constraints.append((j, i))
                        
#             # add constraint the if a tag starts with I then the previous tag should be the same entity
#             # the for example I-CONDITION cant be followed by I-VAL and vice versa
#             # if self.index_to_tag[i.item()][0] == 'I':
#             #     for j in one_indices:
#             #         if self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
#             #             constraints.append((i, j))
#             #             constraints.append((j, i))
                        

#         # add constraints for the tags that are not related to the intent at all
#         for i in zero_indices:
#             for j in zero_indices:
#                 constraints.append((i, j))


#         return constraints

#     def __init_hidden(self):
#         return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2)) # initialize hidden state

#     def __get_lstm_features(self, sentence, intent):
#         self.hidden = self.__init_hidden()
#         word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)
#         # print("Word Embedding Shape:", word_embeddings.shape) # (len(sentence), batch_size, embedding_dimension)
#         intent_embeddings = self.intent_embedding(intent).view(1, 1, -1).repeat(len(sentence), 1, 1)
#         # print("Intent Embedding Shape:", intent_embeddings.shape) # (len(sentence), batch_size, embedding_dimension)
#         embeddings = torch.cat((word_embeddings, intent_embeddings), dim=2) 
#         # print("Embedding Shape:", embeddings.shape) # (len(sentence), batch_size, embedding_dimension * 2)
#         lstm_out, self.hidden = self.lstm(embeddings, self.hidden)
#         # print("LSTM Out Shape:", lstm_out.shape) # (len(sentence), batch_size, hidden_dimension)
#         lstm_features = self.hidden_to_tag(lstm_out)
#         # print("LSTM Features Shape:", lstm_features.shape) # (len(sentence), batch_size, output_dimension (number of target entities)) -> number of entities 3andena 30 
#         return lstm_features

#     def neg_log_likelihood(self, sentence, tags, intent, mask):
#         # print("Intent Shape:", intent.shape)
#         emissions = self.__get_lstm_features(sentence, intent) # (sentence_length, batch_size, number of tags) # emissions where each word in the sequence coressponds to probability for tag
#         # print("NEG LIKE FEATURES:", features.shape)
#         # print(features)
#         # mask = mask.view(1, -1)
#         # print("Mask Shape:", mask.shape)
#         # repeated_mask = mask.repeat(features.shape[0], 1, 1).contiguous()
#         # print(repeated_mask.shape)
#         # print(repeated_mask)
#         # features = features * mask
#         # print("Intent:", intent[0])
#         # print("Mask:", mask)
#         # print("Features:", features)
#         # print("Features Shape:", features.shape)
#         # mask = mask.view(-1, 1)
#         # print("Emissions[0] before:", emissions[0])
#         # print("Mask:", mask)
#         intent_mask = torch.where(mask == 0, torch.tensor([INF]), torch.tensor([0.0]))
#         # print("Log Mask:", intent_mask)
#         # intent_mask = torch.tensor([1 if i not in torch.where(mask == 0)[0] else INF for i, mask in enumerate(mask)], dtype=torch.long)
#         # intent_mask = intent_mask.view(1, 1, -1)
#         # print("Intent Mask:", intent_mask)
#         emissions = emissions + intent_mask
#         # print("Mask Shape:", intent_mask.shape)
#         # print("Emissions Shape", emissions.shape)
#         # print("Emissions[0] after:", emissions)
#         tags = tags.view(-1, 1)
#         loss = -self.crf(emissions, tags)
#         # print("Loss:", loss)
#         return loss

#     def forward(self, sentence, intent, mask):
#         emissions = self.__get_lstm_features(sentence, intent)
#         # print(lstm_features)
#         # print("LSTM Features", lstm_features)
#         # print("Forward LSTM features:", lstm_features.shape)
#         # repeated_mask = mask.repeat(lstm_features.shape[0], 1, 1).contiguous()
#         # test_mask = torch.ones(len(sentence), 1, self.output_dim)
#         # print("Repeated Mask Shape:", repeated_mask.shape)
#         # print("Mask Shape:", mask.shape)
#         # lstm_features = lstm_features * mask
#         # print("Forward LSTM features:", lstm_features.shape)
#         # print(self.crf.transitions)
#         print(mask)
#         intent_mask = torch.where(mask == 0, torch.tensor([INF]), torch.tensor([0.0]))
#         emissions = emissions + intent_mask
#         constraints = self.__create_constraints(mask, intent)
#         tag_sequence = self.crf.decode(emissions, constraints)
#         # print("Tag Sequence:", tag_sequence)
#         return tag_sequence
    

In [80]:
INF = -100

class ConstrainedCRF(CRF):
    def __init__(self, num_tags):
        super(ConstrainedCRF, self).__init__(num_tags)

    def decode(self, emissions: torch.Tensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]], mask: Optional[torch.ByteTensor] = None) -> List[List[int]]:
        """Find the most likely tag sequence using Viterbi algorithm.

        Args:
            emissions:  
            mask:  
            constraints: List of tuples containing the constraints for the CRF.

        Returns:
            List of list containing the best tag sequence for each batch.
        """
        self._validate(emissions, mask=mask)
        if mask is None:
            mask = emissions.new_ones(emissions.shape[:2], dtype=torch.uint8)

        if self.batch_first:
            emissions = emissions.transpose(0, 1)
            mask = mask.transpose(0, 1)

        return self._viterbi_decode(emissions, mask, constraints)

    def _viterbi_decode(self, emissions: torch.FloatTensor, mask: torch.ByteTensor, constraints: List[Tuple[torch.IntTensor, torch.IntTensor]]) -> List[List[int]]:
        print("Viterbi Decode")
        '''
            override the viterbi decode function to include the constraints
        '''
        assert emissions.dim() == 3 and mask.dim() == 2  # (sequence_length, batch_size, num_tags)
        assert emissions.shape[:2] == mask.shape
        assert emissions.size(2) == self.num_tags
        assert mask[0].all()

        # get the sequence length and batch size
        sequence_length, batch_size = mask.shape

        # first we apply the constraints to the transitions
        constrained_transitions = self.transitions.clone()
        constrainted_satrt_transitions = self.start_transitions.clone()
        for constraint in constraints:
            if constraint[0] == -1:
                # start transition constraint
                constrainted_satrt_transitions[constraint[1]] = INF
            else:
                constrained_transitions[constraint[0], constraint[1]] = INF

        # start transition and first emission
        # tensor of size (batch_size, num_tags)
        # constraints for start transition
        score = constrainted_satrt_transitions + emissions[0]

        # back pointers to store the tag sequences that give the best score
        # keeps tracks of the best paths up to the current timestep
        backpointers = []

        # we start from 1 since the first timestep is already computed
        # using the start transition and the first emission
        for i in range(1, sequence_length):

            # broadcast the current score for every possible next tag
            # the score contains the score of the best path up to the current timestep
            # shape: (batch_size, num_tags, 1)
            broadcast_score = score.unsqueeze(2)

            # emission[i] is the emission score for the current timestep
            # broadcast the emission score for every possible current tag
            # shape: (batch_size, 1, num_tags)
            broadcast_emission = emissions[i].unsqueeze(1)

            # the addition of the broadcasted score and the broadcasted emission and the trainsition score
            # result int the shape (batch_size, num_tags, num_tags) where each entry (i, j, k)
            # the score for transitioning from tag j to tag k at the current timestep
            # and emitting k
            next_score = broadcast_score + constrained_transitions + broadcast_emission

            # Find the maximum score over all possible current tag
            # shape: (batch_size, num_tags) the maximum score for each possible current tag to update the score
            # indices: (batch_size, num_tags) the index of the tag that gives the maximum score
            next_score, indices = next_score.max(dim=1)

            # update the score and the backpointers
            score = torch.where(mask[i].unsqueeze(1), next_score, score)

            # add the indices to the backpointers
            backpointers.append(indices)

        # End transition score
        # shape: (batch_size, num_tags)
        score += self.end_transitions

        # Now that we have the final score, we can trace back the best path

        # shape: (batch_size,) containing the length of each sequence
        seq_ends = mask.long().sum(dim=0) - 1

        # shape: (batch_size, num_tags) containing the score of the best tag for each sequence
        # in our case will only have one since our batch size is 1
        best_tags_list = []

        # loop on tags for each sequence in the batch (again 1)
        for idx in range(batch_size):

            # find the best tag for the last timestep
            _, best_last_tag = score[idx].max(dim=0)
            best_tags = [best_last_tag.item()]

            # backtrace the best tag for each timestep
            # we move from the last timestep to the first
            for hist in reversed(backpointers[:seq_ends[idx]]):
                best_last_tag = hist[idx][best_tags[-1]]
                best_tags.append(best_last_tag.item())

            # reverse the order
            best_tags.reverse()

            # again we only have one sequence in the batch
            best_tags_list.append(best_tags)

        return best_tags_list


class NERModel(nn.Module):
    def __init__(self, vocab_size, word_embedding_dim, intent_embedding_dim, hidden_dim, output_dim, number_of_intents, index_to_tag):

        super(NERModel, self).__init__()
        # hyperparameters
        self.word_embedding_dim = word_embedding_dim
        self.inten_embedding_dim = intent_embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.output_dim = output_dim
        self.index_to_tag = index_to_tag

        # model layers
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.intent_embedding = nn.Embedding(number_of_intents, intent_embedding_dim)
        self.lstm = nn.LSTM(word_embedding_dim + intent_embedding_dim,hidden_dim // 2, bidirectional=True, dropout=0.4)
        self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
        self.crf = ConstrainedCRF(output_dim)

    def __create_constraints(self, mask, intent):
        constraints = []
        one_indices = torch.where(mask == 1)[0]  # tags that are for the intent
        # tags that are not for the intent
        zero_indices = torch.where(mask == 0)[0]

        # print(one_indices)
        # print(zero_indices)

        for i in one_indices:
            # add constraints for the tags that are related to the intent
            # the transition from an intent tag to a non-intent tag is not allowed
            for j in zero_indices:
                constraints.append((i, j))
                constraints.append((j, i))

            # constraints that the tag if O can't be followed by I
            # but I can be followed by O
            # if self.index_to_tag[i.item()] == 'O':
            #     for j in one_indices:
            #         if self.index_to_tag[j.item()][0] == 'I':
            #             constraints.append((i, j))

            # add constraints that if the tag begins with B then the next tag can't begin with B of the same
            # if tag begins with B then it can't be followed by B of the same entity
            # or I of a different entity
            # it can be followed by I of same entity, O or B of a different entity
            # add constraints that if the tag begins with B then the next tag cant begin with I of a different entity
            # if self.index_to_tag[i.item()][0] == 'B':
            #     constraints.append((i, i))
            #     for j in one_indices:
            #         if self.index_to_tag[j.item()][0] == 'I' and self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
            #             constraints.append((i, j))

            # create start constraints
            # if the tag starts with I then it can't be the first tag
            if self.index_to_tag[i.item()][0] == 'I':
                constraints.append((-1, i))
                # for j in one_indices:
                    # if self.index_to_tag[j.item()][2:] != self.index_to_tag[i.item()][2:]:
                    #     constraints.append((i, j))
                    #     constraints.append((j, i))

        # add constraints for the tags that are not related to the intent at all
        for i in zero_indices:
            for j in zero_indices:
                constraints.append((i, j))
                constraints.append((j, i))

        return constraints

    def __init_hidden(self):
        '''
            Initialize the hidden state of the LSTM
        '''
        return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2))

    def __get_emissions(self, sentence, intent):
        # initialize the hidden state for the lstm
        # the hidden state is initialized for each input sequence
        # to prevent cascading information from on input sequence to the other
        # as the sentences are independent
        # this also helps prevent overfitting of the model
        self.hidden = self.__init_hidden()

        # generate the word embedding for each word in the input sequence
        # captures the semantic information of the sentence
        word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)

        # generate the intent embedding for the sequence intent
        # to sway the model to choose from certain tags based on the inten
        # by learning semantic information about the intent
        # since there is a single intent for each sentence but we concatenate it with the word embedding
        # we repeat the embedding with the length of the sequence
        intent_embeddings = self.intent_embedding(intent).view(1, 1, -1).repeat(len(sentence), 1, 1)

        # the final embeddings are concatenated to be passed to the lstm layer
        embeddings = torch.cat((word_embeddings, intent_embeddings), dim=2)
        # 
        lstm_out, self.hidden = self.lstm(embeddings, self.hidden)

        lstm_features = self.hidden_to_tag(lstm_out)

        return lstm_features

    def negative_log_likelihood(self, sentence, tags, intent, mask):

        emissions = self.__get_emissions(sentence, intent)

        intent_mask = torch.where(mask == 0, torch.tensor([INF]), torch.tensor([0.0]))

        emissions = emissions + intent_mask

        tags = tags.view(-1, 1)
        loss = -self.crf(emissions, tags)
        return loss

    def forward(self, sentence, intent, mask):
        # get the emission scores from the BiLSTM
        # the emission score represent the score that a word is a certain tag
        # the score is calculated based on the word embedding and the intent embedding
        emissions = self.__get_emissions(sentence, intent)

        # since we are using the negative log likelihood loss
        # map the 0s to -inf and the 1s to 0
        # as the scores could be are negative
        intent_mask = torch.where(
            mask == 0, torch.tensor([INF]), torch.tensor([0.0]))

        # adjust the emissions to account for the intent mask
        # the emission score for the tags that are not related to the intent are set to -inf
        emissions = emissions + intent_mask

        # baseed on the mask and intent create the constraints
        # the constraints are used to guide the CRF to select the best tag sequence
        # by adjusting the transition scores from one tag to another
        constraints = self.__create_constraints(mask, intent)

        # decode the best tag sequence using the CRF
        # using the emissions and the constraints
        tag_sequence = self.crf.decode(emissions, constraints)

        return tag_sequence


### **HELPER FUNCTIONS**

In [81]:
# constants
WORD_EMBEDDING_DIM = 50
INTENT_EMBEDIING_DIM = 50
HIDDEN_DIM = 64

In [82]:
def read_dataset():
    data = pd.read_csv('../ner_dataset/ner_dataset.csv', encoding='latin1')

    # remove white spaces from column names
    data.columns = data.columns.str.strip()

    print(data.columns)
    # Group by 'Sentence #' and aggregate
    grouped_data = data.groupby('Sentence #').agg({
        'Word': lambda x: ''.join(x),  # Join words into a single sentence
        'Tag': lambda x: list(x.str.strip()),       # Collect tags into a list
        'Intent': lambda x: list(x.str.strip().str.replace('_', ' '))     # Collect intents into a list
    }).reset_index()  # Reset index to make 'Sentence #' a regular column

    return data, grouped_data


def prepare_data(dataframe):
    dataset = []
    # a dictionary with the format 
    # intent: [[sentences], [tags]]
    other_dataset = {} # this will be spearated by intent to split the data to train and validation for each intent
    i = 0
    for _, row in dataframe.iterrows():
        sentence = row['Word'][1:]
        tags = row['Tag']
        intents = row['Intent']
        dataset.append((sentence, tags, intents[0]))

        if intents[0] not in other_dataset:
            other_dataset[intents[0]] = [[sentence], [tags]]
        else:
            other_dataset[intents[0]][0].append(sentence)
            other_dataset[intents[0]][1].append(tags)

    return dataset, other_dataset

In [83]:
# Format data
data, goruped_data = read_dataset()
print(data["Intent"].unique())

goruped_data.head()

Index(['Sentence #', 'Word', 'Tag', 'Intent'], dtype='object')
[' assertion' ' assignment operation' ' bitwise operation' ' casting'
 ' class declaration' ' comment' ' conditional operation'
 ' constant declaration' ' file system' ' for loop'
 ' function declaration' ' git operation' ' ide operation' ' input'
 ' interactive commands' ' libraries' ' mathematical operation'
 ' membership operation' ' output' ' variable declaration' ' while loop']


,Sentence #,Word,Tag,Intent
0,0,verify that data is less than or equal Debugg...,"[O, O, B-LHS, O, B-CONDITION, I-CONDITION, I-C...","[assertion, assertion, assertion, assertion, a..."
1,1,check that total amount is equal -606060,"[O, O, B-LHS, I-LHS, O, B-CONDITION, B-RHS]","[assertion, assertion, assertion, assertion, a..."
2,2,check that office location is greater than ob...,"[O, O, B-LHS, I-LHS, O, B-CONDITION, I-CONDITI...","[assertion, assertion, assertion, assertion, a..."
3,3,assert that emergency contact is greater than...,"[O, O, B-LHS, I-LHS, O, B-CONDITION, I-CONDITI...","[assertion, assertion, assertion, assertion, a..."
4,4,validate that category type is more than Neur...,"[O, O, B-LHS, I-LHS, O, B-CONDITION, I-CONDITI...","[assertion, assertion, assertion, assertion, a..."


In [84]:
# training data = (sentence, tags, intent)
training_data, training_validation_data = prepare_data(goruped_data)

print(training_data[0])
print(len(training_data))

print(len(training_validation_data))

('verify that data is less than or equal Debugging Session', ['O', 'O', 'B-LHS', 'O', 'B-CONDITION', 'I-CONDITION', 'I-CONDITION', 'I-CONDITION', 'B-RHS', 'I-RHS'], 'assertion')
2729
21


In [85]:
# split the data to train and validation based on the intent
other_train_data = []
other_validation_data = []

for intent in training_validation_data:
    sentences = training_validation_data[intent][0]
    tags = training_validation_data[intent][1]
    split_index = math.floor(len(sentences) * 0.8)
    other_train_data.extend([(sentences[i], tags[i], intent) for i in range(split_index)])
    other_validation_data.extend([(sentences[i], tags[i], intent) for i in range(split_index, len(sentences))])

print(len(other_train_data))
print(len(other_validation_data))

assert len(training_data) == len(other_train_data) + len(other_validation_data)


2180
549


### **PREPARE DATA**

In [86]:
# convert intent to index
with open('../ner_dataset/annotations/annotations.json', 'r') as f:
    data = json.load(f)

annotations = data['annotations']

intents = list(annotations.keys())

print(intents)
print(len(intents))

intent_to_index = {intent: index for index, intent in enumerate(intents)}

# intent_to_index["<UNK>"] = len(intent_to_index)

['assertion', 'assignment operation', 'bitwise operation', 'casting', 'class declaration', 'comment', 'conditional operation', 'constant declaration', 'file system', 'for loop', 'function declaration', 'git operation', 'ide operation', 'input', 'interactive commands', 'libraries', 'mathematical operation', 'membership operation', 'output', 'variable declaration', 'while loop']
21


In [87]:
# convert data to indices
word_to_index = {}
tag_to_index = {}
for sentence, tags, intents in training_data:
    for word in sentence.split():
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)
            
    for tag in tags:
        if tag not in tag_to_index:
            tag_to_index[tag] = len(tag_to_index)

# add the 'UNK' token
word_to_index['<UNK>'] = len(word_to_index)

tag_to_index['<UNK>'] = len(tag_to_index)

tag_to_index

{'O': 0,
 'B-LHS': 1,
 'B-CONDITION': 2,
 'I-CONDITION': 3,
 'B-RHS': 4,
 'I-RHS': 5,
 'I-LHS': 6,
 'B-OPERATOR': 7,
 'B-OPERAND': 8,
 'I-OPERAND': 9,
 'I-OPERATOR': 10,
 'B-VAR': 11,
 'I-VAR': 12,
 'B-TYPE': 13,
 'B-CLASS': 14,
 'I-CLASS': 15,
 'I-COMMENT': 16,
 'B-COMMENT': 17,
 'B-LOG': 18,
 'B-VAL': 19,
 'I-VAL': 20,
 'B-ACTION': 21,
 'I-ACTION': 22,
 'B-DIR': 23,
 'B-START': 24,
 'B-END': 25,
 'B-STEP': 26,
 'B-COLLECTION': 27,
 'I-COLLECTION': 28,
 'B-FUNC': 29,
 'I-FUNC': 30,
 'B-PARAM': 31,
 'I-PARAM': 32,
 'B-MESSAGE': 33,
 'I-MESSAGE': 34,
 'B-LINE': 35,
 'B-FILE': 36,
 'B-LIB_NAME': 37,
 'B-ELEMENT': 38,
 'I-ELEMENT': 39,
 '<UNK>': 40}

In [88]:
vocab_size = len(word_to_index)

print("Vocabulary Size: ", vocab_size)

number_of_tags = len(tag_to_index)

print("Number of Tags: ", number_of_tags)

number_of_intents = len(intent_to_index)

print("Number of Intents: ", number_of_intents)

Vocabulary Size:  1035
Number of Tags:  41
Number of Intents:  21


In [89]:
tag_to_index
index_to_tag = {index: tag for tag, index in tag_to_index.items()}
index_to_tag

{0: 'O',
 1: 'B-LHS',
 2: 'B-CONDITION',
 3: 'I-CONDITION',
 4: 'B-RHS',
 5: 'I-RHS',
 6: 'I-LHS',
 7: 'B-OPERATOR',
 8: 'B-OPERAND',
 9: 'I-OPERAND',
 10: 'I-OPERATOR',
 11: 'B-VAR',
 12: 'I-VAR',
 13: 'B-TYPE',
 14: 'B-CLASS',
 15: 'I-CLASS',
 16: 'I-COMMENT',
 17: 'B-COMMENT',
 18: 'B-LOG',
 19: 'B-VAL',
 20: 'I-VAL',
 21: 'B-ACTION',
 22: 'I-ACTION',
 23: 'B-DIR',
 24: 'B-START',
 25: 'B-END',
 26: 'B-STEP',
 27: 'B-COLLECTION',
 28: 'I-COLLECTION',
 29: 'B-FUNC',
 30: 'I-FUNC',
 31: 'B-PARAM',
 32: 'I-PARAM',
 33: 'B-MESSAGE',
 34: 'I-MESSAGE',
 35: 'B-LINE',
 36: 'B-FILE',
 37: 'B-LIB_NAME',
 38: 'B-ELEMENT',
 39: 'I-ELEMENT',
 40: '<UNK>'}

In [90]:
# Assertions
i = 0
for sentence, tags, intents in training_data:
    if len(sentence.split()) != len(tags):
        print(f"Example {i}: Sentence Length: {len(sentence.split())}, Tags Length: {len(tags)}")
        print("Example:", training_data[i])
        # sentence += chr(random.randint(65, 90))
        # other_train_data[i] = (sentence, tags, intents)
    i +=1

for sentence, tags, intents in training_data:
    assert len(sentence.split()) == len(tags)

In [91]:
# get the tags for each intent
with open('../ner_dataset/intent_to_tags.json', 'w') as intent_to_tags_file:
    intent_to_tags = {}

    for file in os.listdir('../ner_dataset/annotations/final_annotations'):
        with open(f'../ner_dataset/annotations/final_annotations/{file}', 'r') as f:
            data = json.load(f)
            # print(data)
            tags = data['classes']
            tags.append('O')
            # print(tags)
            intent = file.split('.')[0].replace('_', ' ')
            intent_to_tags[intent] = tags
            # print(intent_to_tags[intent])

    json.dump(intent_to_tags, intent_to_tags_file, indent=4)


all_tags = list(tag_to_index.keys())

### **INTENTS HANDLING**

In [92]:
# here we want to filter out the tags that are not relevent to the given intent
def create_mask(intent, all_tags, intent_to_tags):
    intent_tags = intent_to_tags[intent]
    final_tags = []


    # create BI tags for the intent
    for tag in intent_tags:
        # print(tag)
        if tag == 'O': 
            final_tags.append(tag)
            continue
        
        final_tags.append('B-' + tag)
        final_tags.append('I-' + tag)

    mask = [tag in final_tags for tag in all_tags]
        
    mask = torch.tensor(mask, dtype=torch.long)
    return mask

mask = create_mask('variable declaration', all_tags, intent_to_tags)
print(mask)
print(len(mask))

tensor([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
41


### **MODEL DEFINITION**

In [98]:
ner_model = NERModel(vocab_size=len(word_to_index), word_embedding_dim=WORD_EMBEDDING_DIM, intent_embedding_dim=INTENT_EMBEDIING_DIM, hidden_dim=HIDDEN_DIM, output_dim=number_of_tags, number_of_intents=number_of_intents, index_to_tag=index_to_tag)
ner_optimizer = optim.SGD(ner_model.parameters(), lr=0.01, weight_decay=1e-4)

In [99]:
# Check predictions before training
precheck_tag = prepare_sequence(other_train_data[0][1], tag_to_index)
print(precheck_tag)

mask = create_mask(other_train_data[0][2], all_tags, intent_to_tags)

print(mask)

with torch.no_grad():
    precheck_sent = prepare_sequence(other_train_data[0][0].split(), word_to_index)
    intent = other_train_data[0][2]
    # precheck_intent = prepare_sequence(intent[0], intent_to_index)
    precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
    print(ner_model(precheck_sent, precheck_intent, mask))

tensor([0, 0, 1, 0, 2, 3, 3, 3, 4, 5])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Viterbi Decode
[[2, 6, 0, 0, 0, 2, 3, 2, 2, 2]]


C:\Users\yazmi\AppData\Local\Temp\ipykernel_4188\1977826129.py:86: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ..\aten\src\ATen\native\TensorCompare.cpp:519.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


In [100]:
final_tags = {}
for intent in intent_to_tags.keys():
    final_tags[intent] = []
    for tag in intent_to_tags[intent]:
        if tag == 'O': 
            final_tags[intent].append(tag)
            continue
        
        final_tags[intent].append('B-' + tag)
        final_tags[intent].append('I-' + tag)   

final_tags

distinct_tags = final_tags.values()

distinct_tags = set([tag for sublist in distinct_tags for tag in sublist])

distinct_tags

print(len(distinct_tags))

tags_to_index = {tag: index for index, tag in enumerate(distinct_tags)}

tags_to_index

53


{'I-DIR': 0,
 'B-ARRAY': 1,
 'I-CLASS': 2,
 'O': 3,
 'I-VAL': 4,
 'I-COLLECTION': 5,
 'I-OPERATION': 6,
 'I-PARAM': 7,
 'I-OPERAND': 8,
 'I-ELEMENT': 9,
 'B-CONDITION': 10,
 'B-RHS': 11,
 'B-FILE': 12,
 'B-COLLECTION': 13,
 'B-OPERATION': 14,
 'B-CLASS': 15,
 'B-PARAM': 16,
 'B-OPERAND': 17,
 'I-RHS': 18,
 'I-ARRAY': 19,
 'B-LIB_NAME': 20,
 'I-LINE': 21,
 'B-ELEMENT': 22,
 'B-DIR': 23,
 'I-LHS': 24,
 'I-MESSAGE': 25,
 'I-END': 26,
 'I-STEP': 27,
 'B-END': 28,
 'B-MESSAGE': 29,
 'I-START': 30,
 'B-LHS': 31,
 'I-LOG': 32,
 'B-ACTION': 33,
 'B-OPERATOR': 34,
 'I-OPERATOR': 35,
 'B-STEP': 36,
 'I-FILE': 37,
 'I-LIB_NAME': 38,
 'B-FUNC': 39,
 'B-VAR': 40,
 'B-TYPE': 41,
 'I-COMMENT': 42,
 'I-VAR': 43,
 'I-TYPE': 44,
 'B-LOG': 45,
 'I-FUNC': 46,
 'B-LINE': 47,
 'B-COMMENT': 48,
 'I-ACTION': 49,
 'I-CONDITION': 50,
 'B-START': 51,
 'B-VAL': 52}

In [101]:
intent_to_index

{'assertion': 0,
 'assignment operation': 1,
 'bitwise operation': 2,
 'casting': 3,
 'class declaration': 4,
 'comment': 5,
 'conditional operation': 6,
 'constant declaration': 7,
 'file system': 8,
 'for loop': 9,
 'function declaration': 10,
 'git operation': 11,
 'ide operation': 12,
 'input': 13,
 'interactive commands': 14,
 'libraries': 15,
 'mathematical operation': 16,
 'membership operation': 17,
 'output': 18,
 'variable declaration': 19,
 'while loop': 20}

In [102]:
# split the data into training and testing
# shuffle the data
from sklearn.model_selection import train_test_split


# random.shuffle(training_data)

# split the data
train_data, test_data = train_test_split(training_data, test_size=0.2, random_state=42, shuffle=True)

In [103]:
# training loop
def train(model, optimizer, training_data, epochs=10):
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for sentence, tags, intent in training_data:
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Step 2. Get our inputs ready for the network, that is,
            # turn them into Tensors of word indices.
            intent_mask = create_mask(intent, all_tags, intent_to_tags)

            sentence = prepare_sequence(sentence.split(), word_to_index)
            target_tags = torch.tensor([tag_to_index[t] for t in tags], dtype=torch.long)
            intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)


            # Step 3. Run our forward pass.
            loss = model.negative_log_likelihood(sentence, target_tags, intent, intent_mask)

            total_loss += loss.item()

            # Step 4. Compute the loss, gradients, and update the parameters by
            loss.backward()
            optimizer.step()

        
        print(f"Epoch: {epoch}, Loss: {total_loss / len(training_data)}")


train(ner_model, ner_optimizer, training_data, epochs=100)

  1%|          | 1/100 [00:19<32:09, 19.49s/it]

Epoch: 0, Loss: 5.128842393615474


  2%|▏         | 2/100 [00:39<32:38, 19.98s/it]

Epoch: 1, Loss: 2.4169054608198257


  3%|▎         | 3/100 [00:59<32:02, 19.81s/it]

...

Epoch: 97, Loss: 0.08071009848047063


 99%|█████████▉| 99/100 [36:08<00:21, 21.58s/it]

Epoch: 98, Loss: 0.06253043562992888


100%|██████████| 100/100 [36:30<00:00, 21.91s/it]

Epoch: 99, Loss: 0.05649604811341652


In [104]:
# Check predictions before training
precheck_tag = prepare_sequence(other_train_data[0][1], tag_to_index)
print(precheck_tag)

mask = create_mask(other_train_data[0][2], all_tags, intent_to_tags)

print(mask)

with torch.no_grad():
    precheck_sent = prepare_sequence(other_train_data[0][0].split(), word_to_index)
    intent = other_train_data[0][2]
    # precheck_intent = prepare_sequence(intent[0], intent_to_index)
    precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
    print(ner_model(precheck_sent, precheck_intent, mask))

tensor([0, 0, 1, 0, 2, 3, 3, 3, 4, 5])
tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
Viterbi Decode
[[0, 0, 1, 0, 2, 3, 3, 3, 4, 5]]


In [105]:
def predict(model, test_data):
    predictions = [] 
    with torch.no_grad():
        for sentence, _, intent in test_data:
            precheck_sent = prepare_sequence(sentence.split(), word_to_index)
            # precheck_intent = prepare_sequence(intent, intent_to_index)
            precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
            mask = create_mask(intent, all_tags, intent_to_tags)
            predictions.append(model(precheck_sent, precheck_intent, mask)[0])
    
    return predictions

In [106]:
# validation data
print(len(other_validation_data))
predictions = predict(ner_model, other_validation_data)

549
Viterbi Decode


Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi De

In [107]:
labels = []
predicted_tags = []

for index, prediction in enumerate(predictions):
    print("Test Example:", index)
    mapped_tags = [all_tags[tag] for tag in prediction]
    predicted_tags.append(mapped_tags)
    labels.append(other_validation_data[index][1])
    print("Sentence:", other_validation_data[index][0])
    print("Actual tags:", other_validation_data[index][1])
    print("Predicted tags:", mapped_tags)

Test Example: 0
Sentence: validate gross income equal gross income
Actual tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'O', 'O']
Predicted tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'O', 'O']
Test Example: 1
Sentence: check is verified more than is verified
Actual tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'I-CONDITION', 'O', 'O']
Predicted tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'I-CONDITION', 'O', 'O']
Test Example: 2
Sentence: validate options greater than options
Actual tags: ['O', 'B-RHS', 'B-CONDITION', 'I-CONDITION', 'O']
Predicted tags: ['O', 'B-RHS', 'B-CONDITION', 'I-CONDITION', 'O']
Test Example: 3
Sentence: verify user preferences equal user preferences
Actual tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'O', 'O']
Predicted tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'O', 'O']
Test Example: 4
Sentence: ensure default config equals default config
Actual tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'O', 'O']
Predicted tags: ['O', 'B-RHS', 'I-RHS', 'B-CONDITION', 'O'

In [108]:
# evaluation 
def evaluate(y_true, y_pred):
    mlb = MultiLabelBinarizer()
    y_true_binarized = mlb.fit_transform(y_true)
    y_pred_binarized = mlb.transform(y_pred)
    
    f1 = f1_score(y_true_binarized, y_pred_binarized, average='weighted')
    precision = precision_score(y_true_binarized, y_pred_binarized, average='weighted')
    recall = recall_score(y_true_binarized, y_pred_binarized, average='weighted')
    accuracy = accuracy_score(y_true_binarized, y_pred_binarized)
    
    return f1, precision, recall, accuracy

f1, precision, recall, accuracy = evaluate(labels, predicted_tags)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Accuracy:", accuracy)

F1 Score: 0.9997597427803581
Precision: 1.0
Recall: 0.99951992318771
Accuracy: 0.9981785063752276


In [109]:
torch.save(ner_model.state_dict(), '../models/final_constrained_ner_model.pth')

In [110]:
ner_model.eval()
with torch.no_grad():
        for sentence, _, intent in test_data:
                precheck_sent = prepare_sequence(sentence.split(), word_to_index)
                # precheck_intent = prepare_sequence(intent, intent_to_index)
                precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
                mask = create_mask(intent, all_tags, intent_to_tags)
                predictions.append(ner_model(precheck_sent, precheck_intent, mask)[0])
    

Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi De

In [112]:
test_model = NERModel(vocab_size=len(word_to_index), word_embedding_dim=WORD_EMBEDDING_DIM, intent_embedding_dim=INTENT_EMBEDIING_DIM, hidden_dim=HIDDEN_DIM, output_dim=number_of_tags, number_of_intents=number_of_intents, index_to_tag=index_to_tag)
test_model.load_state_dict(torch.load('../models/final_constrained_ner_model.pth'))
test_model.eval()
with torch.no_grad():
    for sentence, _, intent in test_data:
            precheck_sent = prepare_sequence(sentence.split(), word_to_index)
            # precheck_intent = prepare_sequence(intent, intent_to_index)
            precheck_intent = torch.tensor([intent_to_index[intent]], dtype=torch.long)
            mask = create_mask(intent, all_tags, intent_to_tags)
            predictions.append(ner_model(precheck_sent, precheck_intent, mask)[0])

Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi Decode
Viterbi De

KeyboardInterrupt: 